In [ ]:
from PIL import Image

def crop_bottom_half(image):
    """
    Przycina obraz do dolnej połowy.
    :param image: Obiekt PIL Image.
    :return: Obiekt PIL Image zawierający dolną połowę oryginalnego obrazu.
    """
    width, height = image.size
    return image.crop((0, height // 4, width, height))


In [1]:
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np

# Model ResNet-18 jako ekstraktor cech
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Identity()  # Wyłączenie ostatniej warstwy
model = model.to(device)
model.eval()

# Transformacje danych
transform = transforms.Compose([
    #transforms.Lambda(crop_bottom_half),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Wczytaj dane podłogi
floor_dataset = ImageFolder("/workspace/JETANK/data/mapping/prepared_2/", transform=transform)
floor_loader = DataLoader(floor_dataset, batch_size=16, shuffle=False)
print(len(floor_dataset))



#Wyekstrahuj embeddings
floor_embeddings = []
with torch.no_grad():
    for inputs, _ in floor_loader:  # Ignorujemy etykiety
        inputs = inputs.to(device)
        embeddings = model(inputs).cpu().numpy()
        floor_embeddings.extend(embeddings)

floor_embeddings = np.array(floor_embeddings)
print(len(floor_embeddings))
# Oblicz średni embedding (wzorzec podłogi)
floor_prototype = np.mean(floor_embeddings, axis=0)
np.save("../models/floor_prototype_full.npy", floor_prototype)  # Zapis wzorca
print("Średni embedding podłogi został zapisany.")


cuda
1768
1768
Średni embedding podłogi został zapisany.
